In [127]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.svm import SVR

# Importing the dataset
df = pd.read_excel('../TrainDataset2024.xls', sheet_name='Sheet1')


In [128]:
# df.info()
# df.describe()
df.head()

,ID,pCR (outcome),RelapseFreeSurvival (outcome),Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,TRG002174,1,144.0,41.0,0,0,0,1,3,3,...,0.517172,0.375126,3.325332,0.002314,3880771.500,473.464852,0.000768,0.182615,0.030508,0.000758
1,TRG002178,0,142.0,39.0,1,1,0,0,3,3,...,0.444391,0.444391,3.032144,0.005612,2372009.744,59.459710,0.004383,0.032012,0.001006,0.003685
2,TRG002204,1,135.0,31.0,0,0,0,1,2,1,...,0.534549,0.534549,2.485848,0.006752,1540027.421,33.935384,0.007584,0.024062,0.000529,0.006447
3,TRG002206,0,12.0,35.0,0,0,0,1,3,3,...,0.506185,0.506185,2.606255,0.003755,6936740.794,46.859265,0.005424,0.013707,0.000178,0.004543
4,TRG002210,0,109.0,61.0,1,0,0,0,2,1,...,0.462282,0.462282,2.809279,0.006521,1265399.054,39.621023,0.006585,0.034148,0.001083,0.005626


In [129]:
## Regression - ignore pCR (outcome)
y = df['RelapseFreeSurvival (outcome)']
X_id = df['ID']
X = df.drop(['RelapseFreeSurvival (outcome)', 'ID'], axis=1)

In [130]:
X.head()

,pCR (outcome),Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,LNStatus,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,1,41.0,0,0,0,1,3,3,1,1,...,0.517172,0.375126,3.325332,0.002314,3880771.500,473.464852,0.000768,0.182615,0.030508,0.000758
1,0,39.0,1,1,0,0,3,3,1,1,...,0.444391,0.444391,3.032144,0.005612,2372009.744,59.459710,0.004383,0.032012,0.001006,0.003685
2,1,31.0,0,0,0,1,2,1,1,0,...,0.534549,0.534549,2.485848,0.006752,1540027.421,33.935384,0.007584,0.024062,0.000529,0.006447
3,0,35.0,0,0,0,1,3,3,1,1,...,0.506185,0.506185,2.606255,0.003755,6936740.794,46.859265,0.005424,0.013707,0.000178,0.004543
4,0,61.0,1,0,0,0,2,1,1,0,...,0.462282,0.462282,2.809279,0.006521,1265399.054,39.621023,0.006585,0.034148,0.001083,0.005626


In [131]:
missing_val_strat = 'fill_with_median'

In [141]:
import numpy as np
# outlier detectors
def percentile_based_outlier(data, threshold=95):
  diff = (100 - threshold) / 2.0
  minval, maxval = np.percentile(data, [diff, 100 - diff])
  return (data < minval) | (data > maxval)

# median absolute deviation
def detect_outliers_mad(df, threshold=3):
    medians = df.median()
    mean_abs_dev = (df - medians).abs().median()
    mod_z_score = (0.6745 * (df - medians) / mean_abs_dev)
    return np.abs(mod_z_score) > threshold

# std deviation based
def std_div_outlier(data, threshold=3):
  std = data.std()
  mean = data.mean()
  is_outlier = []
  for v in data:
    is_outlier.append(True if (v > mean + threshold * std) | (v < mean - threshold * std) else False)
  return is_outlier

def vote(data):
  x = percentile_based_outlier(data)
  y = detect_outliers_mad(data)
  z = std_div_outlier(data)
  temp = list(zip(data.index, x, y, z))
  final = []
  for i in range(len(temp)):
    if temp[i].count(False) >= 2:
      final.append(False)
    else:
      final.append(True)
  return final

In [154]:
X.head()

,pCR (outcome),Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,LNStatus,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,1,41.0,0,0,0,1,3,3,1,1,...,0.517172,0.375126,3.325332,0.002314,3880771.500,473.464852,0.000768,0.182615,0.030508,0.000758
1,0,39.0,1,1,0,0,3,3,1,1,...,0.444391,0.444391,3.032144,0.005612,2372009.744,59.459710,0.004383,0.032012,0.001006,0.003685
2,1,31.0,0,0,0,1,2,1,1,0,...,0.534549,0.534549,2.485848,0.006752,1540027.421,33.935384,0.007584,0.024062,0.000529,0.006447
3,0,35.0,0,0,0,1,3,3,1,1,...,0.506185,0.506185,2.606255,0.003755,6936740.794,46.859265,0.005424,0.013707,0.000178,0.004543
4,0,61.0,1,0,0,0,2,1,1,0,...,0.462282,0.462282,2.809279,0.006521,1265399.054,39.621023,0.006585,0.034148,0.001083,0.005626


In [169]:
# Preprocessing
# 1. fill missing values with median
median = X.median()
X.replace({col: {999: median[col]} for col in X.columns}, inplace=True)

#2 fill outlier with median
outliers = X.apply(detect_outliers_mad)

# count of outliers
outliers.head()

df[outliers] = np.nan
print(df.isna().sum())
df.fillna(median, inplace=True)

print(df.isna().sum())


ID                                 0
pCR (outcome)                     84
RelapseFreeSurvival (outcome)      0
Age                                0
ER                               181
                                ... 
original_ngtdm_Busyness           43
original_ngtdm_Coarseness         85
original_ngtdm_Complexity          3
original_ngtdm_Contrast           62
original_ngtdm_Strength           78
Length: 121, dtype: int64
ID                               0
pCR (outcome)                    0
RelapseFreeSurvival (outcome)    0
Age                              0
ER                               0
                                ..
original_ngtdm_Busyness          0
original_ngtdm_Coarseness        0
original_ngtdm_Complexity        0
original_ngtdm_Contrast          0
original_ngtdm_Strength          0
Length: 121, dtype: int64


In [163]:
## normalization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Xs = scaler.fit_transform(X)

In [164]:
from sklearn.decomposition import PCA
feature_names = list(X.columns)
pca = PCA(n_components=10)
Xs_pca = pca.fit_transform(Xs)
Xs_pca=Xs_pca[:,0:3]

In [175]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

Xs_train, Xs_test, y_train, y_test = train_test_split(Xs_pca, y.values.ravel(), test_size=0.2, random_state=1)

svr_model = SVR(C=0.1, kernel='rbf')
svr_model.fit(Xs_train, y_train.ravel())

# Step 5: Make predictions and evaluate the model
y_pred = svr_model.predict(Xs_test)

svr_mse = mean_squared_error(y_test, y_pred)
svr_r2 = r2_score(y_test, y_pred)

compare = zip(y_test.astype(float), y_pred.astype(float))

from pprint import pprint
pprint(list(compare)[:20])
print('mes: {:.2f}'.format(svr_mse))
print('r2 {:.2f}'.format(svr_r2))

[(np.float64(47.5), np.float64(55.757027248294065)),
 (np.float64(74.0), np.float64(56.42158112655706)),
 (np.float64(34.0), np.float64(55.66949859606099)),
 (np.float64(30.0), np.float64(55.546072557015755)),
 (np.float64(32.0), np.float64(55.1805790501219)),
 (np.float64(77.25), np.float64(56.55465073196237)),
 (np.float64(59.0), np.float64(56.27125327085717)),
 (np.float64(38.0), np.float64(55.527339869830385)),
 (np.float64(105.0), np.float64(56.09176060347774)),
 (np.float64(41.0), np.float64(55.08679003881233)),
 (np.float64(59.0), np.float64(55.50100129917066)),
 (np.float64(63.666666666666664), np.float64(55.42578261685148)),
 (np.float64(31.0), np.float64(55.72820792971678)),
 (np.float64(34.0), np.float64(56.37853396539028)),
 (np.float64(69.0), np.float64(55.512191399473544)),
 (np.float64(50.0), np.float64(55.631713074000906)),
 (np.float64(63.666666666666664), np.float64(55.957487622580196)),
 (np.float64(135.0), np.float64(56.171892772758824)),
 (np.float64(84.0), np.floa

In [177]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(max_depth=4)
tree_reg.fit(Xs_train, y_train)

y_test_pred = tree_reg.predict(Xs_test)

print('Test MSE: ', mean_squared_error(y_test, y_test_pred))
print('Test R2: ', r2_score(y_test, y_test_pred))

Test MSE:  863.4068233977383
Test R2:  -0.12017979503569487
